# Import Packages and DataFrame

In [261]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

In [262]:
# Read DataFrame
df = pd.read_csv('../csv/Hotel_Reviews.csv')

In [263]:
# Checking for null values in the latitude and longetude column
df.isna().sum()

Hotel_Address                                    0
Additional_Number_of_Scoring                     0
Review_Date                                      0
Average_Score                                    0
Hotel_Name                                       0
Reviewer_Nationality                             0
Negative_Review                                  0
Review_Total_Negative_Word_Counts                0
Total_Number_of_Reviews                          0
Positive_Review                                  0
Review_Total_Positive_Word_Counts                0
Total_Number_of_Reviews_Reviewer_Has_Given       0
Reviewer_Score                                   0
Tags                                             0
days_since_review                                0
lat                                           3268
lng                                           3268
dtype: int64

In [264]:
# Dropping new values
df.dropna(inplace=True)

In [265]:
# DataFrame shape
df.shape

(512470, 17)

# Geopy

## Create new column with latitude and longitude

In [266]:
# Creating a new column with the lat and lng column as a string
df['geom'] = df['lat'].map(str) + ',' + df['lng'].map(str)

In [267]:
# Saving the unique coordinates into a new variable
unique_geom = df['geom'].unique()

In [268]:
# Turning the array into a DataFrame
unique_geom_df = pd.DataFrame(unique_geom)
unique_geom_df[0]

0           52.36057589999999,4.915968299999999
1        51.49188779999999,-0.19497060000000002
2               51.5137335,-0.10875119999999999
3       51.514218400000004,-0.18090320000000001
4                  48.874348100000006,2.2897334
                         ...                   
1467                      48.1935235,16.3669145
1468              48.2104468,16.371578099999997
1469            51.5020912,-0.15777950000000002
1470             51.499981,-0.19287910000000003
1471                      48.2037451,16.3356767
Name: 0, Length: 1472, dtype: object

In [269]:
# Creating a function to get the city from the coordinates
geolocator = Nominatim(user_agent="geoapiExercises")
def city(coord):
    location = geolocator.reverse(coord, exactly_one=True)
    address = location.raw['address']
    city = address.get('city', '')
    return city

# Creating a function to get the country from the coordinates
def country(coord):
    location = geolocator.reverse(coord, exactly_one=True)
    address = location.raw['address']
    state = address.get('country', '')
    return state

In [270]:
# Applying the funtion to the unique coordinates DataFrame and create a city column
unique_geom_df['city'] = unique_geom_df[0].apply(city)

In [271]:
# Applying the funtion to the unique coordinates DataFrame and get the country column
unique_geom_df['country'] = unique_geom_df[0].apply(country)

In [273]:
# Setting index to the coordinates DataFrame
unique_geom_df.set_index(0, inplace=True)

In [275]:
# Setting coordinates to become the index
df.set_index('geom', inplace=True)

In [276]:
# Merge the unique coordinates to df using the index 
pd.merge(df, unique_geom_df, left_index=True, right_index=True, how='outer')
df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,city,country
"41.328375799999996,2.1129637999999997",Calle K s n Entrada principal Mercabarna Sants...,258,8/2/2017,7.8,Hotel Best Western PLUS Alfa Aeropuerto,United States of America,The hotel wasn t close to the metro so we had...,32,2521,Great price for what I got Rooms were extreme...,12,1,9.2,"[' Leisure trip ', ' Group ', ' Twin Room with...",1 days,41.328376,2.112964,Barcelona,España
"41.328375799999996,2.1129637999999997",Calle K s n Entrada principal Mercabarna Sants...,258,8/2/2017,7.8,Hotel Best Western PLUS Alfa Aeropuerto,Bangladesh,Far from the city,6,2521,Restaurant was cozy Staffs were very friendly...,45,1,7.1,"[' Leisure trip ', ' Family with young childre...",1 days,41.328376,2.112964,Barcelona,España
"41.328375799999996,2.1129637999999997",Calle K s n Entrada principal Mercabarna Sants...,258,7/24/2017,7.8,Hotel Best Western PLUS Alfa Aeropuerto,Thailand,The guest are very noisy the room have no sou...,27,2521,Comfy bed near airport free shuttle bus,9,27,5.4,"[' Leisure trip ', ' Group ', ' Twin Room with...",10 days,41.328376,2.112964,Barcelona,España
"41.328375799999996,2.1129637999999997",Calle K s n Entrada principal Mercabarna Sants...,258,7/23/2017,7.8,Hotel Best Western PLUS Alfa Aeropuerto,Australia,Restaurant was extremely ordinary so have tap...,28,2521,Perfect location if you need to fly out early...,32,11,9.2,"[' Leisure trip ', ' Couple ', ' Double Room w...",11 days,41.328376,2.112964,Barcelona,España
"41.328375799999996,2.1129637999999997",Calle K s n Entrada principal Mercabarna Sants...,258,7/22/2017,7.8,Hotel Best Western PLUS Alfa Aeropuerto,Spain,It was definitely design by a man Lightning w...,26,2521,It had free airport shuttle,7,1,9.2,"[' Leisure trip ', ' Solo traveler ', ' Twin R...",12 days,41.328376,2.112964,Barcelona,España
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"52.4001813,4.893665299999999",Ndsm Plein 28 Amsterdam Noord 1033 WB Amsterda...,170,9/1/2015,8.2,DoubleTree by Hilton Hotel Amsterdam NDSM Wharf,United States of America,Huge 3 day concert right behind the hotel Mus...,15,1593,Close to ferry Nice personnel,6,1,3.8,"[' Leisure trip ', ' Couple ', ' Queen Guest R...",702 day,52.400181,4.893665,Amsterdam,Nederland
"52.4001813,4.893665299999999",Ndsm Plein 28 Amsterdam Noord 1033 WB Amsterda...,170,9/1/2015,8.2,DoubleTree by Hilton Hotel Amsterdam NDSM Wharf,United Kingdom,The noise from the festival we were not told ...,16,1593,No Positive,0,7,6.3,"[' Leisure trip ', ' Group ', ' Twin Guest Roo...",702 day,52.400181,4.893665,Amsterdam,Nederland
"52.4001813,4.893665299999999",Ndsm Plein 28 Amsterdam Noord 1033 WB Amsterda...,170,8/31/2015,8.2,DoubleTree by Hilton Hotel Amsterdam NDSM Wharf,Germany,No Negative,0,1593,personal very friendly,4,17,8.8,"[' Group ', ' Queen Guest Room with River View...",703 day,52.400181,4.893665,Amsterdam,Nederland
"52.4001813,4.893665299999999",Ndsm Plein 28 Amsterdam Noord 1033 WB Amsterda...,170,8/25/2015,8.2,DoubleTree by Hilton Hotel Amsterdam NDSM Wharf,United Kingdom,No Negative,0,1593,Great location by tghe Amstell river and free...,17,4,10.0,"[' Leisure trip ', ' Couple ', ' Queen Guest R...",709 day,52.400181,4.893665,Amsterdam,Nederland


In [278]:
# Exporting
df.to_csv('../csv/hotel_review_city_state.csv')